In [ ]:
# IO
import glob
from pathlib import Path
try:
    import cPickle as pickle
except ModuleNotFoundError:
    import pickle

# Utility Libraries
import math
from datetime import datetime
import re
import csv
import itertools

# Data Processing
import pandas as pd
import numpy as np
from bcpn_pipeline import data, features, models, consts
import shap

# Viz
%matplotlib inline
import matplotlib as mpl
from matplotlib.dates import DateFormatter
from matplotlib.cbook import boxplot_stats
import matplotlib.dates as mdates
import matplotlib.transforms as mtrans
import seaborn as sns
sns.set_style("whitegrid")

import matplotlib.pyplot as plt
plt.rcParams.update(
    {'figure.autolayout': True, 
    }
)
# plt.rcParams.update({'figure.facecolor': [1.0, 1.0, 1.0, 1.0]})

# configure autoreloading of modules
%load_ext autoreload
%autoreload 2


# Load Results

In [ ]:
pred_res = []
for f in consts.OUTPUT_PATH_PRED.glob('*_pred.csv'):
    df = pd.read_csv(f)
    pred_res.append(df)
    
pred_res = pd.concat(pred_res, axis=0).reset_index(drop=True)
pred_res.drop(columns=['Unnamed: 0'], inplace=True)
pred_res

In [ ]:
auc_res = []
for f in consts.OUTPUT_PATH_PRED.glob('*_auc.csv'):
    df = pd.read_csv(f)
    auc_res.append(df)
    
auc_res = pd.concat(auc_res, axis=0).reset_index(drop=True)
auc_res.drop(columns=['Unnamed: 0'], inplace=True)
auc_res

In [ ]:
roc_res = []
for f in consts.OUTPUT_PATH_PRED.glob('*_roc.csv'):
    df = pd.read_csv(f)
    roc_res.append(df)
    
roc_res = pd.concat(roc_res, axis=0).reset_index(drop=True)
roc_res.drop(columns=['Unnamed: 0'], inplace=True)
roc_res

In [ ]:
auc_res['type'] = 'test'
roc_res['type'] = 'test'

In [ ]:
# Get aggregate results (including mean, std, and variance) across runs, by featureset and method
agg_res = None
pred_res_agg = pd.DataFrame()
groupby_cols = ['featureset', 'method', 'features_selected', 'tuned', 'type']
metrics = ['accuracy', 'precision', 'sensitivity', 'specificity', 'f1_score']

for metric in metrics:
    agg_funcs = ['mean', 'std', 'var']
    if metric != 'f1_score':
        pred_res[metric] = pred_res[metric] * 100 # Scale to be reported as a percentage
    df = pred_res.groupby(groupby_cols)[metric].agg(agg_funcs).reset_index()
    df.rename(columns={col: f'{metric}_{col}' for col in agg_funcs}, inplace=True)

    if pred_res_agg.empty:
        pred_res_agg = df
    else:
        pred_res_agg = pred_res_agg.merge(df, on=groupby_cols)

pred_res_agg = pred_res_agg.merge(auc_res[['auc_mean', 'auc_std'] + groupby_cols], on=groupby_cols, how='outer')
pred_res_agg.fillna(-1, inplace=True)
# pred_res_agg.to_csv(Path.joinpath(consts.OUTPUT_PATH_PRED, 'pred_agg.csv'))
pred_res_agg

In [ ]:
# Format column stats as mean +- std
# Note that train AUC was not obtained, so it will be -1 +- (-1), as expected after filling nans

for metric in metrics + ['auc']:
    pred_res_agg.rename(columns={f'{metric}_mean': metric}, inplace=True)
    pred_res_agg[metric] = pred_res_agg.apply(
        lambda x: '%0.2f $\pm$ %0.2f' % (x[metric], x[metric + '_std']),
        axis=1
    )
pred_res_agg = pred_res_agg[groupby_cols + metrics + ['auc']]
# pred_res_agg.to_csv(Path.joinpath(consts.OUTPUT_PATH_PRED, 'pred_agg_condensed.csv'))
pred_res_agg

In [ ]:
roc_res

In [ ]:
# Create legend labels for ROC curve plotting (pull these over from auc_res)

merge_cols =  [col for col in roc_res.columns if '_mean' not in col and '_std' not in col] 
roc_res = roc_res.merge(auc_res, on=merge_cols)

roc_res['legend_label'] = roc_res.apply(
    lambda x: '%s (AUC = %0.2f $\pm$ %0.2f)' % (x['method'], x['auc_mean'], x['auc_std']),
    axis=1
)
roc_res

# ROC Curves

In [ ]:
featuresets = list(roc_res.featureset.unique())
methods = list(roc_res.method.unique())

fs_titles = {fs: None for fs in featuresets}
for fs in fs_titles.keys():
    title = 'Next-' 
    if 'day' in fs:
        title = title + 'Day'
    elif 'week' in fs:
        title = title + 'Week'
    elif 'month' in fs:
        title = title + 'Month'
    
    title = title + ' Prediction w/' + ('Full' if 'all_scores' in fs else 'MEMS-Only') + ' Feature Set'
    fs_titles[fs] = title
fs_titles

In [ ]:
featuresets

## Optimized methods

In [ ]:
for fs in featuresets:
    print(fs)
    plt.figure(figsize=(9,5))
    df = roc_res[(roc_res['featureset'] == fs) & (roc_res['tuned'] ==  True) & (roc_res['features_selected'] == True)]
    g = sns.lineplot(x='fpr_mean', y='tpr_mean', hue='legend_label', data=df)
    g.plot([0, 1], [0, 1], linestyle='--', lw=2, color='black',
            label='Chance', alpha=.8)
        
    g.set(xlabel='False Positive Rate (Positive Label: 1)', 
          ylabel='True Positive Rate (Positive Label: 1)')
    g.legend(title='Model',
             bbox_to_anchor=(1.05, 1), ncol=1)
    plt.savefig(Path.joinpath(consts.OUTPUT_PATH_PRED, f'roc_curves_optimized_{fs}.png'))
    plt.show()

In [ ]:
featuresets

# Feature Importance

In [ ]:
# # Individual graphs
# for fs in featuresets:
#     n_lags = pred_res[pred_res['featureset'] == fs]['n_lags'].iloc[0]
        
#     '''5-repeated 5-fold cross validation
#     Feature importance calculated for tuned classifiers and test sets only
#     Iterate through all runs and all folds to get feature importance graphs'''
#     for method, run, fold in [(method, run, fold) for method in methods for run in range(0, 5) for fold in range(0,5)]:
#         values = None
#         f_png = None
        
#         # Get first match (loop results in only one file)
#         for f in consts.OUTPUT_PATH_PRED.glob(f'shap_values_{fs}_{method}*_tuned_run_{run}_fold_{fold}.pkl'):
#             f_png = Path.joinpath(f.parent, f'{f.stem}.png')
#             values = pickle.load(open(f, 'rb'))
#             break
        
#         if method == 'RF' or method == 'SVM': # Get results for positive class only
#             values = values[:, :, 1]

#         shap.plots.bar(values, show=False)
# #         plt.savefig(f_png)
#         plt.show()

In [ ]:
plt.rcParams.update({'font.size': 20})

In [ ]:
for fs in featuresets:
    
    n_lags = pred_res[pred_res['featureset'] == fs]['n_lags'].iloc[0]
    
    '''5-repeated 5-fold cross validation
    Feature importance calculated for tuned classifiers and test sets only
    Iterate through all runs and all folds to get feature importance graphs'''
    for method in methods:
        sv_all = {}
        for run, fold in [(run, fold)for run in range(0, 5) for fold in range(0,5)]:
            feats = None
            sv = None

            # Get first match (loop results in only one file)
            for f in consts.OUTPUT_PATH_PRED.glob(f'feats_{fs}_{method}*_tuned_run_{run}_fold_{fold}.pkl'):
                feats = pickle.load(open(f, 'rb'))
                break

            for f in consts.OUTPUT_PATH_PRED.glob(f'shap_values_{fs}_{method}*_tuned_run_{run}_fold_{fold}.pkl'):
                sv = pickle.load(open(f, 'rb'))
                break

            if method == 'RF' or method == 'SVM': # Get results for positive class only
                sv = sv[:, :, 1]

            for i, feat in enumerate(feats):
                if (sv_feat := sv_all.get(feat)) is not None:
                    sv_all[feat] = None

                sv_feat_curr = sv[:, i].values

                if sv_feat is not None: # If current feature already has shap values
                    sv_feat_curr = np.concatenate((sv_feat, sv_feat_curr)) # Tack the current ones on to the existing

                # Update the dictionary of all shap values
                sv_all.update({feat: sv_feat_curr})

            
        # Get the mean absolute value of all shap values for each feature
        sv_all = {k: round(np.abs(v).mean(), 2) for k, v in sv_all.items()}

        # Sort the dictionary
        sv_all = {k: v for k, v in sorted(sv_all.items(), key=lambda item: item[1], reverse=True)}

        # Plot the dictionary (follow Lundberg's approach for summary plot
        df = pd.DataFrame(sv_all, index=['mean(|SHAP value|)']).T
        df.index.rename('feature', inplace=True)
        df.reset_index(inplace=True)
        df = df.iloc[:8, :]
        
        fig, ax = plt.subplots(figsize=(14, 8))
        bars = ax.barh('feature', 'mean(|SHAP value|)', 0.7, align='center', color='teal', data=df)
        ax.bar_label(bars, fmt='+ %g', label_type='edge', padding=5)

        # plt.yticks('feature', fontsize=13)
        plt.xlabel('mean(|SHAP value|)')
#         plt.ylabel('feature')
        plt.gca().invert_yaxis()
        plt.gca().spines['right'].set_visible(False)
        plt.gca().spines['top'].set_visible(False)
        plt.gca().spines['left'].set_visible(False)
        plt.savefig(f'results/washout/prediction_task/shap_all_{fs}_{method}.png', bbox_inches='tight')
        plt.show()
        

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
bars = ax.barh('feature', 'mean(|SHAP value|)', 0.7, align='center', color='teal', data=df)
ax.bar_label(bars, fmt='+ %g', label_type='edge', padding=5)

# plt.yticks('feature', fontsize=13)
plt.xlim(0, 0.55)
plt.xlabel('mean(|SHAP value|)')
plt.ylabel('feature')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
with open('results/prediction_task/feats_study_week_XGB_2_lags_max_depth_1_tuned_run_3_fold_4.pkl', 'rb') as f:
    feats = pickle.load(f)
feats

In [ ]:
with open('results/prediction_task/shap_values_study_week_XGB_2_lags_max_depth_1_tuned_run_3_fold_4.pkl', 'rb') as f:
    sv = pickle.load(f)
sv.values

In [ ]:
np.abs(np.concatenate([sv[:, 1].values, sv[:, 1].values])).mean()

In [ ]:
[x for x in [y for y in sv[2].values]]

In [ ]:
sv_all['n_events (t-2)']

In [ ]:
f = {feat: {'value': val} for feat, val in dict.fromkeys(feats).items()}
f

In [ ]:
feats_d = {}
for i in range(0, len(feats)):
    feats_d[feats[i]] = {
        'value': None,
        'shap_value': sv[0][i].values
    }
feats_d

# {'shap_value': float(x) for x in [y for y in sv[1].values]}

In [ ]:
feats_d = dict(sorted(feats_d.items(), key=lambda x: abs(x[1].get('shap_value')), reverse=True))
feats_d

In [ ]:
shap.summary_plot(sv[:, 0:3])